In [ ]:
import subprocess

def processFile(filename : str, annif_project_name : str) :
    out = subprocess.run('annif suggest ' + annif_project_name + ' < ' + filename, stdout=subprocess.PIPE, shell=True)
    title = ''
    with open(filename, 'r') as f:
        line = f.readline().strip('\n')
        title = line.split(':')[1].strip() if ':' in line else line
    id = filename.split('/')[-1].replace('.txt', '') + ';' + title +  ';"'
    result = id
    for line in out.stdout.decode('utf-8').splitlines():
        linesplit = line.split('\t')
        if len(linesplit) == 3:
            if len(result) > len(id):
                result += ', '
            result += linesplit[1]
    return result + '"'

In [ ]:
import os
from datetime import datetime

def processDirectory(directory_path: str, annif_projects: list[str]):
    start_dateTime = datetime.now()
    print('START [ ', start_dateTime, ' ] Processando: ', directory_path)
    directory = os.fsencode(directory_path)
    count_files = 0
    result = {}
    for file in os.listdir(directory):
        filename = os.fsdecode(file).split('.')
        if filename[1] == 'txt':
            count_files += 1
            for project in annif_projects:
                if project not in result:
                    result[project] = []
                line = processFile(directory_path + os.fsdecode(file), project) + '\n'
                result[project].append(line)
    end_dateTime = datetime.now()
    print('END [ ', end_dateTime, ' ] Processado: ', directory_path)
    delta_time = end_dateTime - start_dateTime
    print('\t- Delta: ', delta_time)
    secs = int(delta_time.total_seconds())
    dias, secs = divmod(secs, 24 * 3600)
    horas, secs = divmod(secs, 3600)
    minutos, secs = divmod(secs, 60)
    print('\t- Tempo de processamento:', f'{dias} dia(s), {horas} hora(s), {minutos} minuto(s), {secs} segundo(s)')
    print('\t- Tempo de processamento Médio por Arquivo:', delta_time.total_seconds() / count_files, 'segundo(s)')
    return result

In [ ]:
# import os
# # ERRO no nome do Arquivo
# def evalDirectory(directory_path: str, annif_projects: list[str], output_path: str = None):
#     directory = os.fsencode(directory_path)
#     result = {}
#     for file in os.listdir(directory):
#         filename = os.fsdecode(file).split('.')
#         if filename[1] == 'txt':
#             for project in annif_projects:
#                 if project not in result:
#                     result[project] = ''
#                 #line = processFile(directory_path + os.fsdecode(file), project) + '\n'
#                 if output_path != None:
#                     fileoutput = 'ANNIF_' + input_files.split('/')[-2] + '_' + k + '-EVAL.txt'
#                     out = subprocess.run('annif eval ' + project + ' ' + , stdout=subprocess.PIPE, shell=True)
#                     print(out.stdout)
#                 result[project].append(line)
#     return result

In [ ]:
import os

def evalDirectory(directory_path: str, annif_projects: list[str], output_path: str):
    for item in os.listdir(directory_path):
        if os.path.isdir(directory_path + item):
            input_files = directory_path + item + '/'
            for p in annif_projects:
                print('Eval: ' + input_files + ' : ' + p)
                fileoutput = 'ANNIF_' + input_files.split('/')[-2] + '_' + p + '-EVAL.txt'
                out = subprocess.run('annif eval ' + p + ' ' + input_files, stdout=subprocess.PIPE, shell=True)
                with open(fileoutput, 'w') as out_f:
                    out_f.write(out.stdout.decode('utf-8'))